### Initialization of Image Dataset
This file is specifically to make it easy for the user to initialize the image dataset by drawing images of stars from the internet using the hips2fits service.

In [50]:
import os 
import pandas as pd 
from astroquery.hips2fits import hips2fits
from matplotlib.image import imsave, imread
import astropy.coordinates as coord
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [51]:
#Create/Update ImageCount.csv
def createCSV():
    PATH = os.path.join('Images', 'ImageCount.csv')

    DataFrame = pd.DataFrame([[len(os.listdir(os.path.join('Image Dataset')))]], columns=['Number of Images']) #Takes the number of files in the Image Dataset directory and sets that equal to the number of images
    DataFrame.to_csv(PATH)

createCSV()

### Image Setup

The below cell gets images of the stars using the hips2fits service present with astroquery. Because there is no formula for FOV, or how zoomed in the image is, the user is asked to specify the perfect FOV and choose a specific image to save to the Image Dataset folder. This will time you out after a while because of hips2fits' DDOS protections. If that happens, just rerun the below cell. **SOME IMAGES WILL BE AWFUL AND DO NOT ACCURATELY REPRESENT STARS: THESE IMAGES ARE IGNORED BY PLACING THEIR SPECIFIC INDEX IN MODEL.IPYNB**

In [ ]:
#Sets up Images in the Images Folder
def PullData():
    PulledData, Count = pd.read_csv(os.path.join('Images', 'GeneratedData.txt'), index_col=0), pd.read_csv(os.path.join('Images', 'ImageCount.csv'), index_col=0)['Number of Images'][0]

    PulledData = PulledData.iloc[Count:, :] #Pulls Only the rows AFTER what has already been appended to the Image Dataset using iloc

    return Count, PulledData

def Image(ra, dec, FOV):
    result = hips2fits.query(
        hips='CDS/P/DSS2/color',
        width=164,
        height=164,
        ra=coord.Angle(ra, unit=u.hourangle),
        dec=coord.Angle(dec, unit=u.deg),
        fov=coord.Angle(FOV * u.deg),
        projection='TAN',
        format='jpg'
        ) #Queries the DSS2 color HIPS for the image, returned in the form of a matrix.
    
    return result

def UserImage(pulled_row, Count, min_value=0.005, max_value=0.01):
    clear_output() #Clears the Jupyter output for easier viewing and repetition

    ra, dec = pulled_row['Ra'], pulled_row['Dec'] #Extracts the RA and DEC coordinates

    FOVs = np.linspace(min_value, max_value, 4) #Returns 4 values for FOV 
    image_num, matrices = 1, [Image(ra, dec, fov) for fov in FOVs] #Gets 
    figure, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(9, 9), layout='constrained')

    for ax in (ax1, ax2, ax3, ax4):
        ax.imshow(matrices[image_num - 1]) #Shows all the images in all the axes, labels with the image number
        ax.tick_params(labelbottom=False, bottom=False, labelleft=False, left=False)
        ax.set(title=f'Star Image {image_num}')
        image_num += 1
    
    plt.show()
    
    valid_num, exit_triggered = False, False
    while not valid_num: 
        try:
            num_index_to_save = input('What image out of those displayed do you wish to save (exit if you want to exit) (resize if you want to resize)? ') 

            if num_index_to_save.lower() == 'exit':
                valid_num, exit_triggered = True, True #Exits everything
            
            elif num_index_to_save.lower() == 'resize':
                satisfied = False 
                while not satisfied: 
                    min_val, max_val = input('What new minimum value do you want for FOV?'), input('What new maximum value do you want for FOV') #Gets values for resizing the FOV (passed to recursion)

                    try:
                        UserImage(pulled_row, Count, float(min_val), float(max_val)) #Recursion that resizes the FOV and displays again
                        satisfied = True
                    
                    except: 
                        print('Please enter valid floats for resizing. Default min is 0.005, default max is 0.01')
                valid_num = True
            
            else: 
                num_index_to_save = int(num_index_to_save) - 1 #Gets the index to save, saves it to Image Dataset
                imsave(os.path.join('Image Dataset', f'Star{Count + 1}.png'), matrices[num_index_to_save])
                valid_num = True

        except:
            print('You must enter a valid integer!')

    createCSV() #Updates the CSV file for images to make sure no images are added to the dataset twice

    return exit_triggered #This allows the for loop to be broken (See code below)

createCSV()
count, pulled_data = PullData()
for index, row in pulled_data.iterrows(): #Iterates over the rows of the pulled data
    count = PullData()[0] #Updates count
    
    exit_triggered = UserImage(row, count)
        
    if exit_triggered == True: 
        break #If exit is triggered, break out of the for loop

The cell bellow creates the labels for the image dataset based on their spectral class, of whether they are more white, blue, or orange. 

In [52]:
#Create the ImageClassification.csv file/update it 
IMAGE_CLASSIFICATION, SPEC_CLASS, IMAGES = os.path.join('Images', 'ImageClassification.csv'), os.path.join('Dataset', 'Dataset.csv'), os.path.join('Image Dataset')

Stars_Classified, Spec_Class = [], pd.read_csv(SPEC_CLASS, index_col=0)['Spectral Class'] #Find the Spectral Class of all the stars with images
for index in range(len(os.listdir(IMAGES))): #For each image in the Images Dataset
    
    if Spec_Class[index] == 'O': #If the spectral class is O, append 0, which represents blue
        Stars_Classified.append(0)
    
    elif Spec_Class[index] == 'B' or Spec_Class[index] == 'A' or Spec_Class[index] == 'F': #If the spectral class is B, A, or F, append 1, which represents white
        Stars_Classified.append(1)
    
    else:
        Stars_Classified.append(2) #If the spectral class is anything else, append 2, which represents orange

pd.DataFrame(Stars_Classified, columns=['Color']).to_csv(IMAGE_CLASSIFICATION) #Save the file